# 데이터 그리기

In [ ]:
import sys, os
print(os.getcwd())

current_dir = os.path.dirname(os.getcwd())
parent_dir = os.path.dirname(current_dir)
print(current_dir)
sys.path. append(current_dir)

import numpy as np
from dataset.mnist import load_mnist
from PIL import Image
import matplotlib. pyplot as plt

def img_show(img):
    plt.imshow(img, cmap='gray' )
    plt.axis('off') # 축 숨기기
    plt.show()
#pil_img = Image.fromarray(np.uint8(img))
#pil_img.show()

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

img = x_train[178] 
label = t_train [178]
print(label) # 5

print(img.shape) # (784,)
img = img.reshape(28, 28) # 형상을 원래 이미지의 크기로 변형
print(img.shape) # (28, 28)

img_show(img)


# 숫자를 인식하는 코드 만들기 /예측 성능 측정과 정확도 출력 프로그램

In [ ]:
# coding: utf-8

import os, sys, time
print(os.getcwd())
current_dir = os.path.dirname(os.getcwd())
print(current_dir)
os.chdir(current_dir)

import numpy as np
import pickle
from dataset.mnist import load_mnist
from common. functions import sigmoid, softmax

def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

def init_network(): #신경망 초기화 함수
    print(os.getcwd())
    with open("ch02/sample_weight.pkl", 'rb') as f: #sample_weight.pkl 파일에서 신경망의 가중치와 편향 값을 로드합니다
        network = pickle.load(f)
    return network # 초기화된 신경망을 반환환

def predict(network, x): #예측함수
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x,W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1,W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    z3 = sigmoid(a3)
    y = softmax(a3)

    return y
    
x, t = get_data() #함수로 테스트 데이터와 라벨을 로드합니다
network = init_network() #함수로 신경망을 초기화합니다.

#예측 및 정확도 계산산
accuracy_cnt = 0 #변수는 정확하게 예측된 개수를 셉니다
start_time = time.time() #start_time과 end_time을 사용하여 실행 시간을 측정합니다.
time.sleep(1)
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y) #확률이 가장 높은 원소의 인덱스를 얻음
    if p == t[i]:
        print(i, 'Predicted num= ', p, ' Original num= ', t[i])
        #time.sleep(0.5)
        accuracy_cnt += 1 #모든 테스트 데이터에 대해 예측을 수행하고, 예측된 값이 실제 라벨과 일치할 경우 accuracy_cnt를 증가시킵니다.
    #else p!= t[i]:
        #print(i, 'Predicted num= ', p, ' Original num= ', t[i])

end_time = time.time()
print("Accuracy: " + str(float(accuracy_cnt) / len(x))) #최종적으로 정확도와 실행 시간을 출력합니다.
elapsed_time = end_time - start_time
print(f"실행 시간: {elapsed_time} 초")

# 배치함수 사용하기

In [ ]:
import sys, os
from pathlib import Path

print(os.getcwd())
current_dir = os.path.dirname(os.getcwd())
print(current_dir)
os.chdir(current_dir)

import numpy as np
import pickle
from dataset.mnist import load_mnist
from common. functions import sigmoid, softmax

def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

def init_network():
    print(os.getcwd())
    with open("ch02/sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x,W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1,W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    z3 = sigmoid(a3)
    y = softmax(a3)

    return y
    
x, t = get_data()
network = init_network()

batch_size = 100 #배치크기 batch_size를 100으로 설정하여 배치 단위로 데이터를 처리합니다.
accuracy_cnt = 0
start_time = time.time()
time.sleep(1) #1초대기
for i in range(0, len(x), batch_size): #for 반복문을 사용하여 배치 단위로 데이터를 처리하고, 각 배치에 대해 예측을 수행합니다.
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1) #np.argmax 함수를 사용하여 가장 높은 확률을 가진 클래스를 선택합니다.
    accuracy_cnt += np.sum(p == t[i:i+batch_size])
end_time = time.time()
print("Accuracy:" + str(float(accuracy_cnt) / len(x))) #최종 정확도 출력
elapsed_time = end_time - start_time
print(f"실행 시간: {elapsed_time} 초")